In [3]:
import numpy as np
import re
import nltk
from collections import Counter
import os
from pathlib import Path
import datetime
from string import ascii_lowercase
import pandas as pd
import os
from os import listdir
from os.path import isfile, join, isdir
import datetime
import unicodedata
from nltk.corpus import stopwords # Import the stop word list
from string import ascii_lowercase
import re
import logging

In [4]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p: ', level=logging.WARN)

In [5]:
%run ..\_Util.ipynb


## Pré-processamento do conjunto de arquivos textos (Corpus O&G)

#### Script responsável por iterar o conjunto de arquivos texto extraídos dos arquivos PDF e realizar as etapas de pré-processamento para adequá-los aos algoritmos de NLP em que serão aplicados.

As seguintes etapas são realizadas:
    - substituição de caracteres acentuados por sua representação equivalente
    - normalização de todos os caracteres para minúsculos
    - eliminação de stopwords, nos idiomas portuguÊs, inglês e espanhol
    - eliminação de arquivos sem conteúdo
    - unificação de todos os arquivos em um único arquivo para cada corpus
    - reformatação das sentenças, incluindo quebra de linha e considerando caractere de ponto como final de sentença    
    - eliminação de caracteres inválidos e pontuação (são eliminados todos os caracteres não-alfanuméricos, exceto o ponto final e outros previamente mapeados)
    - substituição de tokens numéricos pela tag <NUMBER>
//    - eliminação de ruídos (palavras raras, caracterizadas por ruídos no corpus) 


##### Definições de variáveis

In [6]:
## Definindo parametros sobre localizacao dos arquivos
#pathRaizCorporaEntrada = 'T:\\OpenScience\\Corpora\\2- Textos Extraídos\\Apache Tika\\Publicos'
#pathRaizCorporaSaida   = 'T:\\OpenScience\\Corpora\\3- Textos Preprocessados\\Algoritmos de Embedding - V2\\Públicos'
pathRaizCorporaEntrada = join('..', '..', 'resources', 'corpora', '2- Textos Extraidos')
pathRaizCorporaSaida   = join('..', '..', 'resources', 'corpora', '3- Textos Preprocessados')

# Filtros para iteracao dos arquivos na pasta contendo corpus a ser processado
extensao = "*.txt"
extensaoExcel = ".xls"
extensaoPowerPoint = ".ppt"
pastaAtualCorporaEntrada = "."

# variaveis a serem redefinidas pelas implementacoes especificas
pastaCorporaEntrada = ""
pastaCorporaSaida = pathRaizCorporaSaida
nomeArquivoCorpus = 'corpus.txt'

# armazena o conteudo final do corpus processado, a ser escrito no arquivo de saida
textoCorpus = ""

##### Elementos Parametrizáveis

In [7]:
## Parametros de execucao

convert_lowercase = True
remove_special_chars = True
convert_diacritics = True  # converte diacríticos em seus correspondentes não-acentuados
remove_stopwords = True
remove_numeric_tolens = True
remove_single_char = True  # remove tokens formados por um único caractere
remove_hifen_suffixes = True  # elimina sufixos verbais, do tipo -se, -lo
replace_hyphenation = True

ignoraExcel = False
ignoraPowerPoint = False
pastasParaIgnorar = ['ArquivosNP1']

CARACTERE_VAZIO = ''
CARACTERE_ESPACO = ' '
# Conforme o tipo do conteudo e a origem do corpus, o hifen pode ser substituido por espaco em branco ou eliminado.
# a decisao eh tomada pontualmente a cada corpus, conforme analise exploratoria de observacao do conteudo
tratamento_hifenizacao = CARACTERE_VAZIO

# Define se o algoritmo realizara a reconstrucao das sentenças a partir dos caracteres de delimitação (.?:!). Isso se faz
# particularmente necessário em textos extraídos de PDF, em que podem ocorrer quebras de linha resultantes do processo de extração
# mas em textos nativamente digitais, essa etapa pode ser desconsiderada
isReconstroiSentencas = True
#isNILC = False

In [36]:
def processaArquivosDaPasta(listaCorpora, nomeArquivoSaidaCorpus):
    momentoInicial = datetime.datetime.now()
    tokens_totais = 0
    sentencas_totais = 0

    arquivoSaida = join(pastaCorporaSaida, nomeArquivoSaidaCorpus)
    logging.info('Arquivo de saida sera escrito em: ' + arquivoSaida)

    # armazena o conteudo (ja processado) de cada arquivo
    conteudoPorArquivoProcessado = []
    
    for nomePastaEntradaAtual in listaCorpora:
        # Itera os arquivos de cada pasta, carregando seu conteudo
        pastaCorporaEntrada = join(pathRaizCorporaEntrada, nomePastaEntradaAtual)
        logging.info("\n\n * Processando arquivos da pasta: " + pastaCorporaEntrada)

        if (not os.path.isdir(pastaCorporaEntrada)):
            logging.error('Pasta ',pastaCorporaEntrada,' nao encontrada')

        # iterando subpastas
        #onlyDirs = [d for d in listdir(pastaEntradaAtual) if (isdir(join(pastaEntradaAtual, d)) and d not in pastasParaIgnorar )]
        #for dir in onlyDirs:
        #    processaArquivosDaPasta(pastaEntradaAtual, dir, pastaSaidaAtual)

        arquivosPastaAtual = Path(pastaCorporaEntrada).glob(extensao)

        for path in arquivosPastaAtual:
            path_arquivo = str(path) # because path is object not string
            converteArquivo(path_arquivo, conteudoPorArquivoProcessado)# grava os dados em conteudoPorArquivoProcessado[]    

    # dupla quebra de linha entre os arquivos (necessario ao Bert)
    textoCorpus = '\n\n'.join([arquivo for arquivo in conteudoPorArquivoProcessado])    
    textoCorpus = posProcessamentos(textoCorpus)
        
    gravaArquivo(arquivoSaida, textoCorpus)

    print("\n\n\nTempo total decorrido: ", datetime.datetime.now() - momentoInicial)    
    return textoCorpus

In [35]:
def converteArquivo(arquivoEntrada, conteudoPorArquivoProcessado):
    (filepath, filename) = os.path.split(arquivoEntrada)
    
    if ignoraExcel and extensaoExcel in filename:
        #logging.warn("Ignorando arquivo Excel " + filename)
        return
    
    if ignoraPowerPoint and extensaoPowerPoint in filename:
        #logging.warn("Ignorando arquivo PowerPoint " + filename)
        return
    
    #logging.info("Iniciando leitura do arquivo " + arquivoEntrada)
    texto = leTextoDeArquivo(arquivoEntrada)
    
    #arquivos vazios nao sao processados (threashold 10 caracteres)
    if (len(texto) < 10):
        #logging.warn("... ignorando arquivo vazio: " + filename)
        return 
    
    if (convert_lowercase): texto = text.lower()  
    if (convert_diacritics): texto = eliminaCaracteresAcentuados(texto)
    if (remove_special_chars): texto = eliminaCaracteresEspeciais(texto)
    texto = eliminaStopwords(texto) # a lógica condicional precisou estar dentro do próprio método
    if (remove_numeric_tolens): texto = processaTokensNumericos(texto)
    texto = reconstroiSentencas(texto)  # a lógica condicional precisou estar dentro do próprio método
    if (remove_numeric_tolens): texto = processaTokensNumericos(texto) # executado novamente, para eliminar elementos restantes apos a unificacao de sentencas

    conteudoPorArquivoProcessado.append(texto)
    #gravaArquivo(arquivoSaida, texto)

In [26]:
def eliminaCaracteresAcentuados(texto):
    nfkd_form = unicodedata.normalize('NFKD', texto)
    texto= u"".join([c for c in nfkd_form if not unicodedata.combining(c)])
    return texto

In [15]:
def eliminaCaracteresEspeciais(texto):
    logging.info('Eliminando todos os caracteres especiais, exceto alfanumericos e pontuacao previamente mapeados...')
    texto = texto.replace('\n\n', '\n')   
    texto = texto.replace('-\n', '-')  # corrigindo quebras de linha incorretas, pela posicao do hifen  
    texto = texto.replace('\n', ' \n ')  # adicionando espaco para isolar \n como token
    
#    if not isNILC:
    texto = re.sub('([.,!?()/])', r' \1 ', texto) # Adicionando espaços aos caracteres de pontuacao  # no BERT não separamos os caracteres de pontuação

    texto = re.sub('[^A-Za-z0-9àèìòùÀÈÌÒÙáéíóúÁÉÍÓÚâêîôÂÊÎÔãñõÃÑÕçÇäëïöüÿÄËÏÖÜŸ: .?!&\n\-]+', '', texto) # Eliminando todos os caracteres especiais, mantem apenas quebra de linha, ponto, &, -...
    
    if remove_single_char:
        texto = re.sub(r" [a-zA-Z] ", "", texto) # eliminando acracteres unicos
    
    # tratamento de hinenizacao, conforme parametrizado
    if remove_hifen_suffixes: texto = eliminaSufixosVerbais(texto) # eliminacaode sufixos no contexto do tratamneto de hifenizacao
    if replace_hyphenation: texto = texto.replace('-', tratamento_hifenizacao) # Para o BERT mantém a hifenização
    
    #texto = texto.replace('- ', ' ')  
    #texto = texto.replace(' -', ' ')  # Eliminando tokens compostos por hifen+espaco
    
    return texto

In [16]:
#teste = "asda (acentuação) áéíóúâêì? Pontuação! Mantém. desde 123a 12354x 740m desde 12b 1234c 1000th cesde 740m desde"
#teste = '123a saida reator considera 4-115 variacao temperatura 84-115 dentro faixa 220c 250c perda 8-1 carga <NUMBER> bar 3- reatores tubulares leito fixo conforme dados jagger <NUMBER> 123a '
#teste = re.sub(r'[\d]+\w ', ' <NUMBER> ', teste)
#teste = re.sub(r' \d{1,}-\d{1,} ', ' <RANGE> ', teste)
#print(eliminaCaracteresEspeciais(teste))

asda (acentuação) áéíóúâêì? Pontuação! Mantém. desde 123a 12354x 740m desde 12b 1234c 1000th cesde 740m desde


In [8]:
def eliminaSufixosVerbais(texto):
    texto = texto.replace('-se ', ' ')  
    texto = texto.replace('-lo ', ' ') 
    texto = texto.replace('-os ', ' ') 
    texto = texto.replace('-as ', ' ')     
    #texto = texto.replace(' nao-', ' ') #prefixo não     
    
    return texto

In [38]:
def processaTokensNumericos(texto):
    logging.info('Processando tokens numéricos...')
    
    # Substitui tokens numericos por <NUMBER>
    texto = re.sub(r" \d+ ", " <NUMBER> ", texto)
    texto = re.sub(r' [\d]+\w+ ', ' <NUMBER> ', texto) # Substitui ocorrencias muito comuns do tipo 700m, 2008a, 124o etc
    texto = re.sub(r' \d{1,}-\d{1,} ', ' <NUMBER> ', texto) #Substitui ocorrencias do tipo Range: 118-130

    texto = re.sub(r"( <NUMBER>)+", " <NUMBER>", texto) # substituindo multiplas ocorrencias por apenas uma
    return texto

In [39]:
#teste = "iii 1 a 2 b 3 c 4 d 5 e 6 f 7 g 8 h 9 i 123 asd 123a teste 123. asd 123! asd 12354x aaa 740m aaa 123 aaa 123? aaa 123)"
#teste = " <NUMBER> da Lei 9 "
#teste = '123a saida reator considera 4-115 variacao temperatura 84-115 dentro faixa 220c 250c perda 8-1 carga <NUMBER> bar 3- reatores tubulares leito fixo conforme dados jagger <NUMBER> 123a '
#teste = re.sub(r'[\d]+\w ', ' <NUMBER> ', teste)
#teste = re.sub(r' \d{1,}-\d{1,} ', ' <RANGE> ', teste)
#print(processaTokensNumericos(teste))

iii <NUMBER> a <NUMBER> b <NUMBER> c <NUMBER> d <NUMBER> e <NUMBER> f <NUMBER> g <NUMBER> h <NUMBER> i <NUMBER> asd <NUMBER> teste 123. asd 123! asd <NUMBER> aaa <NUMBER> aaa <NUMBER> aaa 123? aaa 123)


In [15]:
# Mapeando stopwords com NLTK
stopwordsPortugues = stopwords.words("portuguese")
if convert_diacritics: 
    stopwordsPortugues = [eliminaCaracteresAcentuados(t) for t in stopwordsPortugues] #normalizando stopwords
stopwordsIngles = stopwords.words("english")
#stopwordsEspanhol = stopwords.words("spanish")
caracteres_unicos = [c for c in ascii_lowercase ]

# em estudo anterior, obtivemos o conjunto das palavras mais comuns, onde se observam algumas palavras incorretas
# resultantes do processo de extração do arquivo PDF. Estas palavras serao incluídas no vocabulário a ser ignorado
# no corpus final.
# Objetiva-se incrementar esta lista conforme novas palavras sao observadas, a medida em que novos arquivos sao
# acrescentados ao corpus
palavrasIncorretasComuns = ['-', '&', 'ch', 'ii','iii', '\x05', '6leo']

# mais rapido buscar em set do que em list
vocabulario_palavras_ignoradas = set(
                                     stopwordsPortugues + 
                                     stopwordsIngles + 
                                    # stopwordsEspanhol + 
                                     caracteres_unicos + 
                                     palavrasIncorretasComuns)


   
#print("Palavras a serem ignoradas do vocabulario", vocabulario_palavras_ignoradas)


Palavras a serem ignoradas do vocabulario {'can', 'too', 'seremos', 'serei', 'tem', "aren't", 'ii', 'aquele', 'during', 'needn', 'estivemos', 'quem', "she's", 'foi', 'tenho', 'aren', 'was', 'j', 'meu', 'estivesse', 'voces', 'r', 'um', 'formos', 'nor', 'tiver', 'they', 'hasn', 'after', 'which', 'because', 'ourselves', 'estive', 'nosso', 'estivera', 'uma', 'h', 'tiveramos', 'by', 'them', 'minha', 'os', 'fomos', 'both', 'houverao', 'sejamos', 'while', 'esteja', 'to', 'over', 'where', 'aquela', 'houver', 'tambem', 'tivera', 'sem', 'tu', 'ch', 'e', 'a', 'being', "couldn't", "hasn't", 'houvera', "shouldn't", 'qual', '-', 'theirs', 'it', 'seja', 'mustn', 'eles', 'when', 'teve', 'te', 'para', 'tivemos', 'about', 'deles', 'aquilo', 'hadn', 'w', 'tinha', 'her', 'mais', 'houverem', "won't", 'against', 'are', 'teus', 'did', 'da', 'este', 'were', "it's", "should've", 's', 'havemos', 'yours', 'foramos', 'isn', 'themselves', 'down', 'hajam', 'has', "you've", 'aquelas', 'here', 'das', 'ou', 'same', 'd

In [32]:
def eliminaStopwords(texto):
    logging.info('Eliminando stopwords...')
    sentenca_sem_stopwords = ''
    
    vocab_stopwords = vocabulario_palavras_ignoradas
    ## Flag para desativar a eliminacao de stopwords, sem alterar a logica interna dos metodos
    if remove_stopwords == False:
        vocab_stopwords = set()
    
    # Ao reconstruir as sentencas, elimina-se todos os caracteres \n no metodo split, eliminando tambem espacos duplicados, etc
    if isReconstroiSentencas:
        sentenca_sem_stopwords = [w for w in texto.split() if not w in vocab_stopwords]
    else:
        texto = re.sub(' +', ' ', texto) # Elimina espacos duplicados
        sentenca_sem_stopwords = [w for w in texto.split(" ") if not w in vocab_stopwords]
        
    texto = " ".join( sentenca_sem_stopwords ) 
    return texto

In [33]:
#frequenciaMinimaPalavra = 1
#texto = 'teste1 raraA raraB raraC teste teste teste1 aaa aaa bbb bbb'
#word_counts = Counter(texto.split())
#palavras_raras = [w for w in word_counts if word_counts[w] <= frequenciaMinimaPalavra]
#print(palavras_raras)

#print('')
#print(eliminaPalavrasRaras(texto))

In [45]:
#"         teste        ".strip()

'teste'

In [39]:
def processaListaMWEs(arquivoMWEs):
    print('Iniciando leitura dos MWEs')
    texto = leTextoDeArquivo(arquivoMWEs)
    texto = texto.replace(',','')
    lista_mwes = texto.split("\n")
    # ordenar lista_mwes  
    lista_mwes = sorted(lista_mwes, key=lambda s: len(s.split()), reverse=True)
    
    print('Processamento da lista de MWEs finalizada')
    return lista_mwes

In [48]:
def posProcessamentos(texto):
    # Eliminando espaco no inicio de cada linha
    texto = texto.replace('\n ', ' \n')
    
    # Eliminando sentencas curtas (<3 tokens)
    tamanho_minimo_sentecas = 3
    sentences = texto.split('\n')
    texto = "\n".join( [sentence for sentence in sentences if len(sentence.split()) >= tamanho_minimo_sentecas])  
    
    return texto


##### Unificando as quebras de linhas, refazendo as sentenças.

Obervou-se a ocorrencia de muitas quebras de linhas desnecessárias, produto de erros na extração do PDF. As quebras de linha serão todas eliminadas, e posteriormente refeitas considerando o caractere de ponto como final de sentença e quebra de linha.


In [34]:
def reconstroiSentencas(texto):
    texto = texto.replace('?','.').replace('!','.')  #.replace('\n', ' ').replace('\r', '') e realizado abaixo:

    if not isReconstroiSentencas:
        logging.info("Ignorando recontrucao de sentenças")
        return texto
        
    logging.info('Reunificando quebras de linha...')
    
    texto = ' '.join(texto.split())  # removendo multiplos espaços consecutivos e outras quebras de linha
    lista_com_quebra_de_linha = texto.split('.')
    
    # unificando caso a sentença contenha ao menos 3 palavras
#    texto = "\n".join( lista_com_quebra_de_linha) 
#    texto = "\n".join( [sentenca for sentenca in lista_com_quebra_de_linha if len(sentenca.split()) > 2]) # 'if' migrado para o pos processamento
    texto = "\n".join( [sentenca for sentenca in lista_com_quebra_de_linha]) 
    
    # eliminando sentencas com menos de 3 palavras
    # lista_com_quebra_de_linha = texto.split('\n')
    # texto = '\n'.join([sentenca for sentenca in lista_com_quebra_de_linha if len(sentenca.split(' ')) > 3])
    
    return texto